In [23]:
import os
import shutil
import nibabel as nib
import numpy as np
import torch
import torchio as tio
from scipy.ndimage import binary_dilation
import pyvista as pv
from Image_Resizing import resize_images_to_moving_reference
from Image_Normalization import save_array_to_nifti1, apply_processing_to_img_folder

In [3]:
atlas_mask = nib.load("E:\\Data_Booster\\Atlas_TOF3D\\Atlas_Files\\TOF atlas_segmentation.nii.gz").get_fdata()

In [ ]:
img = nib.load("E:\\Data_Booster\\data_ETIS_781\\SWI\\2018-104_01-10087-D0MR_12_SWI_Images.nii.gz")
atlas_orig = nib.load("E:\\Data_Booster\\Atlas_TOF3D\\Atlas_Files\\TOF atlas_segmentation.nii.gz")
atlas_orig_data = atlas_orig.get_fdata()

In [7]:
print(img.shape, atlas_orig_data.shape)

(480, 640, 80) (308, 384, 200)


In [9]:
test_folder = "E:\\Data_Booster\\Atlas_TOF3D\\Atlas_Files\\Test"

In [ ]:
image_to_resize = tio.LabelMap("E:\\Data_Booster\\Atlas_TOF3D\\Atlas_Files\\TOF atlas_segmentation.nii.gz")
ref_image = tio.ScalarImage("E:\\Data_Booster\\data_ETIS_781\\SWI\\2018-104_01-10087-D0MR_12_SWI_Images.nii.gz")
transforms = tio.ToCanonical(), tio.Resample(target=ref_image)
total_transform = tio.Compose(transforms)
normalized_image = total_transform(image_to_resize)

In [17]:
image_to_resize = tio.LabelMap("E:\\Data_Booster\\Atlas_TOF3D\\Atlas_Files\\TOF atlas_segmentation.nii.gz")
ref_image = tio.ScalarImage("E:\\Data_Booster\\data_ETIS_781\\SWI\\2018-104_01-10087-D0MR_12_SWI_Images.nii.gz")
normalized_image = tio.Resample(target=ref_image)(image_to_resize)

In [38]:
image_to_resize = tio.LabelMap("E:\\Data_Booster\\Atlas_TOF3D\\Atlas_Files\\TOF atlas_segmentation.nii.gz")
image_to_resize.set_data(torch.flip(image_to_resize.data, [3]))
ref_image = tio.ScalarImage("E:\\Data_Booster\\data_ETIS_781\\SWI\\2018-104_01-10087-D0MR_12_SWI_Images.nii.gz")
transforms = tio.ToCanonical(), tio.Resample(target=ref_image)
total_transform = tio.Compose(transforms)
normalized_image = total_transform(image_to_resize)

In [39]:
normalized_image.save(os.path.join(test_folder, "flipped_3.nii.gz"))

In [40]:
image_to_resize.shape

(1, 308, 384, 200)

In [11]:
atlas_mask_homogenized = np.where(atlas_mask > 0, 1, atlas_mask)

In [13]:
def renumber_mask (array):
    return np.where(array > 0, 1, array)

In [9]:
np.unique(atlas_mask, return_counts=True)

(array([0., 1., 2., 3., 4., 5.]),
 array([23617791,     3564,    15482,     8132,     4309,     5122],
       dtype=int64))

In [12]:
np.unique(atlas_mask_homogenized, return_counts=True)

(array([0., 1.]), array([23617791,    36609], dtype=int64))

In [7]:
source_swi_dir = "E:\\Data_Booster\\data_ETIS_781\\SWI"
target_atlas_dir = "E:\\Data_Booster\\Atlas_TOF3D\\Processed_Atlas\\Duplicated_Atlas"

In [11]:
for file in os.listdir(source_swi_dir):
    new_atlas_filename = "_".join(file.split("_")[:2]) + "_Atlas.nii.gz"
    shutil.copyfile("E:\\Data_Booster\\Atlas_TOF3D\\Atlas_Files\\TOF atlas_segmentation.nii.gz", os.path.join(target_atlas_dir, new_atlas_filename))

In [14]:
apply_processing_to_img_folder(renumber_mask, target_atlas_dir, "E:\\Data_Booster\\Atlas_TOF3D\\Processed_Atlas\\Renumbered_Atlas", "")

Processed image  2018-104_01-10087-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10113-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10114-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10116-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10117-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10118-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10120-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10124-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10126-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10128-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10129-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10136-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10151-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10165-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10167-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10170-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10172-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10173-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10174-D0MR_Atlas.

In [23]:
renumbered_dir = "E:\\Data_Booster\\Atlas_TOF3D\\Processed_Atlas\\Renumbered_Atlas"
for file in os.listdir(renumbered_dir):
    os.rename(os.path.join(renumbered_dir, file), os.path.join(renumbered_dir, file[:-8]+".nii.gz"))

In [24]:
resize_images_to_moving_reference("E:\\Data_Booster\\Atlas_TOF3D\\Processed_Atlas\\Renumbered_Atlas", "E:\\Data_Booster\\Atlas_TOF3D\\Processed_Atlas\\Resized_Atlas", source_swi_dir, image_is_label=True, transform_to_canonical=True, interpolation_method="lanczos", modification_string="Resized_Canonical")

Processed image  2018-104_01-10087-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10113-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10114-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10116-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10117-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10118-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10120-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10124-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10126-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10128-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10129-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10136-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10151-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10165-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10167-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10170-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10172-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10173-D0MR_Atlas.nii.gz
Processed image  2018-104_01-10174-D0MR_Atlas.

In [ ]:
resized_dir = "E:\\Data_Booster\\Atlas_TOF3D\\Processed_Atlas\\Resized_Atlas"
mask_img = nib.load(resized_dir+"\\2018-104_01-10087-D0MR_Atlas_Resized_Canonical.nii.gz")
mask_data = mask_img.get_fdata()

In [36]:
dilated_dir = "E:\\Data_Booster\\Atlas_TOF3D\\Processed_Atlas\\Dilated_Atlas"
mask_dilated_img = nib.load(dilated_dir+"\\Dilation_1\\2018-104_02-10626-D0MR_Atlas_Resized_Canonical_Dilated_1.nii.gz")
mask_dilated_data = mask_dilated_img.get_fdata()

In [22]:
def plot_mask_3D (mask_img, mask_array, cmap="BuGn"):
    grid = pv.ImageData()

    grid.dimensions = np.array(mask_array.shape)

    origin = mask_img.affine[:3,3]
    grid.origin = tuple(origin)

    spacing = mask_img.header.get_zooms()
    grid.spacing = spacing

    grid.point_data["values"] = mask_array.flatten(order="F")

    plotter = pv.Plotter()
    opacity = [0,1]
    plotter.add_volume(grid, scalars="values", opacity=opacity, cmap=cmap)

    plotter.add_axes()
    plotter.show()

In [20]:
plot_mask_3D (mask_img, mask_data, cmap="BuGn")

Widget(value='<iframe src="http://localhost:62781/index.html?ui=P_0x19a81c34250_5&reconnect=auto" class="pyvis…

In [10]:
kernel_size = 5
kernel_radius = (kernel_size - 1) / 2

x, y, z = np.ogrid[-kernel_radius:kernel_radius+1, -kernel_radius:kernel_radius+1, -kernel_radius:kernel_radius+1]

kernel_distance = np.sqrt(x**2 + y**2 + z**2)

kernel = (kernel_distance <= kernel_radius).astype(np.float32)

In [33]:
def dilate_array (array, iterations, structure):
    return binary_dilation(array, iterations=iterations, structure=structure).astype("float32")

In [ ]:
mask_data_dilated1 = binary_dilation(mask_data, iterations=1, structure=kernel).astype("float32")

In [38]:
list(range(2,8))

[2, 3, 4, 5, 6, 7]

In [39]:
for dilation_level in range(2,8):
    source_mask_dir = "E:\\Data_Booster\\Atlas_TOF3D\\Processed_Atlas\\Resized_Atlas"
    target_mask_dir = f"E:\\Data_Booster\\Atlas_TOF3D\\Processed_Atlas\\Dilated_Atlas\\Dilation_{dilation_level}"

    apply_processing_to_img_folder (dilate_array, source_mask_dir, target_mask_dir, f"Dilated_{dilation_level}", iterations=dilation_level, structure=kernel)

Processed image  2018-104_01-10087-D0MR_Atlas_Resized_Canonical.nii.gz
Processed image  2018-104_01-10113-D0MR_Atlas_Resized_Canonical.nii.gz
Processed image  2018-104_01-10114-D0MR_Atlas_Resized_Canonical.nii.gz
Processed image  2018-104_01-10116-D0MR_Atlas_Resized_Canonical.nii.gz
Processed image  2018-104_01-10117-D0MR_Atlas_Resized_Canonical.nii.gz
Processed image  2018-104_01-10118-D0MR_Atlas_Resized_Canonical.nii.gz
Processed image  2018-104_01-10120-D0MR_Atlas_Resized_Canonical.nii.gz
Processed image  2018-104_01-10124-D0MR_Atlas_Resized_Canonical.nii.gz
Processed image  2018-104_01-10126-D0MR_Atlas_Resized_Canonical.nii.gz
Processed image  2018-104_01-10128-D0MR_Atlas_Resized_Canonical.nii.gz
Processed image  2018-104_01-10129-D0MR_Atlas_Resized_Canonical.nii.gz
Processed image  2018-104_01-10136-D0MR_Atlas_Resized_Canonical.nii.gz
Processed image  2018-104_01-10151-D0MR_Atlas_Resized_Canonical.nii.gz
Processed image  2018-104_01-10165-D0MR_Atlas_Resized_Canonical.nii.gz
Proces

In [ ]:
plot_mask_3D (mask_img, mask_data_dilated1, cmap="BuGn")

Widget(value='<iframe src="http://localhost:62781/index.html?ui=P_0x19a96eea950_7&reconnect=auto" class="pyvis…

In [31]:
mask_data_dilated7 = binary_dilation(mask_data, iterations=7, structure=kernel).astype("float32")

In [32]:
plot_mask_3D (mask_img, mask_data_dilated7, cmap="BuGn")

Widget(value='<iframe src="http://localhost:62781/index.html?ui=P_0x19abfe49cd0_12&reconnect=auto" class="pyvi…